## Segmenting and Clustering Neighborhoods in Toronto | Part-1  


For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1. Start by creating a new Notebook for this assignment.

2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

PostalCode | Borough | Neighborhood
--- | ---: | ---:
M5G | Downtown Torronto | Central Bay Street
M2H | New York | Hillcrest Village
M4B | East York | Parkview Hill, Woodbine Gardens
M1J | Scarborough | Scarborough Village
M4G |  East York | Leaside
M4M |  East Toronto | Studio District
M1R |  Scarborough | Wexford, Maryvale
M9V |  Etobicoke | South Steeles, Silverstone, Humbergate, Jamest...
M9L |  North York | Humbert Summit
M5V |  Downtown Torronto | CN Tower, King and Spadina, Railway Lands, Har...
M1B | Scarborough | Malvern, Rouge 
M5A | Downtown Toronto | Regent Park, Harbourfront

3. To create the above dataframe:

 - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
 - Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
 - More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
 - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
 - Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
 - In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


4. Submit a link to your Notebook on your Github repository. (10 marks)

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

#### Scraping Wikipedia, creating a Dataframe and Transforming.

Importing libraries

In [1]:
# Importing libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

Scraping List of Postal Codes of Given Wikipedia Page 

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extracting_data = requests.get(url).text
wiki_data = BeautifulSoup(extracting_data, 'lxml')

Converting content of PostalCode HTML table as dataframe

In [3]:
# Column names
column_names = ['Postalcode', 'Borough', 'Neighborhood']

# Adding column to dataframe
toronto = pd.DataFrame(columns=column_names)

content = wiki_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text.strip('\n')
            i = i + 1
        elif i == 1:
            borough = td.text.strip('\n')
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [4]:
# Clean dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i += 1

In [5]:
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(','.join).reset_index()
df

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [6]:
df.describe()

,Postalcode,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M6M,North York,Downsview
freq,1,24,4


Data Cleaning | Drop None rows of df and row which contains 'Not assigned' value | All "Not assigned" will be replace to 'NaN'

In [7]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [8]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df_to = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [10]:
df_to.describe()

,Postalcode,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M6M,North York,Downsview
freq,1,24,4


In [11]:
print(df_to.shape)
df_to.head()

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
df_to.to_csv('toronto_part1.csv', index=False) 